In [ ]:
import numpy as np
import pyvista as pv
from pathlib import Path

def initialize_octahedron():
    """Initialize the vertices and faces of an octahedron."""
    vertices = [
        (1, 0, 0), (-1, 0, 0),
        (0, 1, 0), (0, -1, 0),
        (0, 0, 1), (0, 0, -1)
    ]
    faces = [
        (vertices[0], vertices[2], vertices[4]),
        (vertices[0], vertices[2], vertices[5]),
        (vertices[0], vertices[3], vertices[4]),
        (vertices[0], vertices[3], vertices[5]),
        (vertices[1], vertices[2], vertices[4]),
        (vertices[1], vertices[2], vertices[5]),
        (vertices[1], vertices[3], vertices[4]),
        (vertices[1], vertices[3], vertices[5])
    ]
    return faces

def split_triangle(triangle):
    """Split a triangle into four smaller triangles."""
    P1, P2, P3 = triangle
    # Midpoints of each edge
    P4 = tuple((np.array(P1) + np.array(P2)) / 2)
    P5 = tuple((np.array(P2) + np.array(P3)) / 2)
    P6 = tuple((np.array(P1) + np.array(P3)) / 2)
    
    return [
        (P1, P4, P6),
        (P4, P2, P5),
        (P4, P5, P6),
        (P5, P3, P6)
    ]

def subdivide_faces(faces, subdivisions):
    """Recursively subdivide each triangle face into smaller triangles."""
    for _ in range(subdivisions):
        new_faces = []
        for triangle in faces:
            new_faces.extend(split_triangle(triangle))
        faces = new_faces
    return faces

def project_to_unit_sphere(point):
    """Normalize a point to lie on the unit sphere."""
    x, y, z = point
    norm = np.sqrt(x**2 + y**2 + z**2)
    return (x / norm, y / norm, z / norm)

def normalize_faces(faces):
    """Project each vertex of every face onto the unit sphere."""
    return [
        (project_to_unit_sphere(face[0]),
         project_to_unit_sphere(face[1]),
         project_to_unit_sphere(face[2]))
        for face in faces
    ]

def calculate_area_and_centroid(triangle):
    """Calculate the area and centroid of a triangle on the unit sphere."""
    P1, P2, P3 = map(np.array, triangle)
    
    # Calculate the vectors for two edges
    edge1 = P2 - P1
    edge2 = P3 - P1
    
    # Area calculation (cross product norm gives twice the area of the triangle)
    area = 0.5 * np.linalg.norm(np.cross(edge1, edge2))
    
    # Centroid of the triangle
    centroid = (P1 + P2 + P3) / 3
    centroid = project_to_unit_sphere(centroid)  # Project the centroid to the unit sphere
    
    return area, centroid

def generate_sphere_directions(subdivisions=2):
    """Generate directions by subdividing an octahedron and projecting to a sphere."""
    # Initialize the octahedron and subdivide
    faces = initialize_octahedron()
    faces = subdivide_faces(faces, subdivisions)
    
    # Normalize each face to project onto the unit sphere
    faces = normalize_faces(faces)
    
    # Calculate area and centroid of each face
    directions = []
    total_area = 0
    for face in faces:
        area, centroid = calculate_area_and_centroid(face)
        directions.append((centroid, area))
        total_area += area
    
    # Normalize areas so they sum up to the sphere's surface area (4π for unit sphere)
    scale_factor = 4 * np.pi / total_area
    directions = [(centroid, area * scale_factor) for centroid, area in directions]
    
    return directions

def plot_3d_binary_array(Array):

    """Plot the 3D binary array."""

    structure_mesh = pv.wrap(Array)
    structure_iso = structure_mesh.contour([0.5])  # Isosurface at level 0.5

    pl = pv.Plotter(off_screen=True, window_size=[512,384])
    pl.add_mesh(structure_iso, color=(0.87,0.91,0.91), opacity=1, show_edges=False)
    pl.camera_position = 'xz'
    pl.camera.roll = 0
    pl.camera.elevation = 30
    pl.camera.azimuth = 30
    pl.camera.zoom(1.0)
    pl.add_axes(viewport=(0,0,0.3,0.3))
    # pl.screenshot(FName.parent / (FName.name + '.png'))
    pl.show()

"""
Compute the Mean Intercept Length (MIL) for a 3D binary numpy array in a mirrored direction.

Parameters:
binary_array (np.ndarray): A 3D binary numpy array (0s and 1s).
direction (np.ndarray): Normalized 3D direction vector.
step_size (float): Distance between each ray origin point in the xy-plane.

Returns:
float: MIL value for the specified mirrored direction.
"""

DataPath = Path(r'C:\Users\mathi\OneDrive - Universitaet Bern\FABTIB') / '02_Results/ROIs'
InputROIs = sorted([F for F in Path.iterdir(DataPath) if F.name.endswith('.npy')])
Array = np.load(InputROIs[51])

plot_3d_binary_array(Array)

Directions = generate_sphere_directions(subdivisions=2)

# Calculate corner coordinates of the binary array
Z, Y, X = np.array(Array.shape) - 1
Corners = np.array([[0, 0, 0],
                    [X, 0, 0],
                    [X, Y, 0],
                    [0, Y, 0],
                    [0, 0, Z],
                    [X, 0, Z],
                    [X, Y, Z],
                    [0, Y, Z]])


In [ ]:
def rotate_to_align_with_direction(Points, Direction, Center):
    
    """
    Rotate corners of the binary array to align the z-axis with the given direction vector.
    
    Parameters:
    corners (np.ndarray): Array of corner points of the binary array.
    direction (np.ndarray): Direction vector to align with.
    
    Returns:
    np.ndarray: Rotated corner coordinates.
    """
    
    # Compute axis of rotation (cross product of z-axis and direction vector)
    z_axis = np.array([0, 0, 1])
    rotation_axis = np.cross(z_axis, Direction)
    rotation_angle = np.arccos(np.dot(z_axis, Direction))
    
    if np.allclose(rotation_axis, 0):
        return Corners  # Already aligned with z-axis

    rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
    cos_theta = np.cos(rotation_angle)
    sin_theta = np.sin(rotation_angle)
    ux, uy, uz = rotation_axis

    # Rodrigues' rotation formula for rotation matrix
    R = np.array([
        [cos_theta + ux*ux*(1 - cos_theta), uy*ux*(1 - cos_theta) - uz*sin_theta, uz*ux*(1 - cos_theta) + uy*sin_theta],
        [ux*uy*(1 - cos_theta) + uz*sin_theta, cos_theta + uy*uy*(1 - cos_theta), uz*uy*(1 - cos_theta) - ux*sin_theta],
        [ux*uz*(1 - cos_theta) - uy*sin_theta, uy*uz*(1 - cos_theta) + ux*sin_theta, cos_theta + uz*uz*(1 - cos_theta)]
    ])

    # Rotate corners around their center
    RPoints = np.einsum('ij,jk->ik', Points - Center, R)

    return RPoints + Center

def plot_corners_with_binary_array(Array, Corners, Direction, RCorners):
    
    """Plot binary array with mirrored rotated corners."""

    # Corners bounding box
    Faces = [
                [4, 0, 1, 2, 3],  # Bottom face
                [4, 4, 5, 6, 7],  # Top face
                [4, 0, 1, 5, 4],  # Side face 1
                [4, 1, 2, 6, 5],  # Side face 2
                [4, 2, 3, 7, 6],  # Side face 3
                [4, 3, 0, 4, 7],  # Side face 4
            ]
    
    Faces = np.hstack(Faces)

    # Create the box in PyVista
    Original = pv.PolyData(Corners, Faces)
    Rotated = pv.PolyData(RCorners, Faces)
    
    # Mirrored direction vector
    Zaxis = pv.Arrow(start=np.array(Array.shape)/2,
                     direction=(0,0,1),
                     scale=max(Array.shape)/2,
                     shaft_radius=8/max(Array.shape),
                     tip_radius=12/max(Array.shape))
    Arrow = pv.Arrow(start=np.array(Array.shape)/2,
                     direction=Direction,
                     scale=max(Array.shape)/2,
                     shaft_radius=8/max(Array.shape),
                     tip_radius=12/max(Array.shape))

    pl = pv.Plotter(off_screen=True, window_size=[512,384])
    pl.add_mesh(Original, color=(1.0,0.0,0.0), show_edges=True, opacity=0.25)
    pl.add_mesh(Rotated, color=(0.0,0.0,1.0), show_edges=True, opacity=0.25)
    pl.add_points(Corners, color=(1.0,0.0,0.0), point_size=15, render_points_as_spheres=True)
    pl.add_points(RCorners, color=(0.0,0.0,1.0), point_size=15, render_points_as_spheres=True)
    pl.add_mesh(Zaxis, color=(1.0,0.0,0.0))
    pl.add_mesh(Arrow, color=(0.0,0.0,1.0))
    pl.camera_position = 'xz'
    pl.camera.roll = 0
    pl.camera.elevation = 30
    pl.camera.azimuth = 30
    pl.camera.zoom(1.0)
    pl.add_axes(viewport=(0,0,0.3,0.3))
    # pl.screenshot(FName.parent / (FName.name + '.png'))
    pl.show()


# Rotate corners to align z-axis with mirrored direction
Direction = Directions[13][0]
Center = np.mean(Corners,axis=0)
R_Corners = rotate_to_align_with_direction(Corners, Direction, Center)

plot_corners_with_binary_array(Array, Corners, Direction, R_Corners)


In [ ]:
# Generate grid of ray origins in the xy-plane, spaced by `step_size`
MinC, MaxC = R_Corners.min(axis=0), R_Corners.max(axis=0)
StepSize = 5
Xp = np.arange(0, MaxC[0], StepSize)
Xn = np.arange(-StepSize, MinC[0], -StepSize)
Yp = np.arange(0, MaxC[1], StepSize)
Yn = np.arange(-StepSize, MinC[1], -StepSize)
Xc = np.concatenate([Xn[::-1], Xp]).astype(int)
Yc = np.concatenate([Yn[::-1], Yp]).astype(int)

XGrid, YGrid = np.meshgrid(Xc, Yc)
Grid = np.column_stack((XGrid.ravel(), YGrid.ravel(), np.zeros(XGrid.size, int) + int(round(MinC[2]))))

# Rotate grid back to the original space
def rotate_to_align_with_direction(Points, Direction, Center):
    
    """
    Rotate corners of the binary array to align the z-axis with the given direction vector.
    
    Parameters:
    corners (np.ndarray): Array of corner points of the binary array.
    direction (np.ndarray): Direction vector to align with.
    
    Returns:
    np.ndarray: Rotated corner coordinates.
    """
    
    # Compute axis of rotation (cross product of z-axis and direction vector)
    z_axis = np.array([0, 0, 1])
    rotation_axis = np.cross(z_axis, Direction)
    rotation_angle = np.arccos(np.dot(z_axis, Direction))
    
    if np.allclose(rotation_axis, 0):
        return Corners  # Already aligned with z-axis

    rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
    cos_theta = np.cos(-rotation_angle)
    sin_theta = np.sin(-rotation_angle)
    ux, uy, uz = rotation_axis

    # Rodrigues' rotation formula for rotation matrix
    R = np.array([
        [cos_theta + ux*ux*(1 - cos_theta), uy*ux*(1 - cos_theta) - uz*sin_theta, uz*ux*(1 - cos_theta) + uy*sin_theta],
        [ux*uy*(1 - cos_theta) + uz*sin_theta, cos_theta + uy*uy*(1 - cos_theta), uz*uy*(1 - cos_theta) - ux*sin_theta],
        [ux*uz*(1 - cos_theta) - uy*sin_theta, uy*uz*(1 - cos_theta) + ux*sin_theta, cos_theta + uz*uz*(1 - cos_theta)]
    ])

    # Rotate corners around their center
    RPoints = np.einsum('ij,jk->ik', Points - Center, R)

    return RPoints + Center
R_Grid = rotate_to_align_with_direction(Grid, Direction, Center)

def plot_corners_with_grid(Corners, RCorners, GridPoints, RGridPoints):
    
    """Plot binary array with mirrored rotated corners."""

    # Corners bounding box
    Faces = [
                [4, 0, 1, 2, 3],  # Bottom face
                [4, 4, 5, 6, 7],  # Top face
                [4, 0, 1, 5, 4],  # Side face 1
                [4, 1, 2, 6, 5],  # Side face 2
                [4, 2, 3, 7, 6],  # Side face 3
                [4, 3, 0, 4, 7],  # Side face 4
            ]
    
    Faces = np.hstack(Faces)

    # Create the box in PyVista
    Original = pv.PolyData(Corners, Faces)
    Rotated = pv.PolyData(RCorners, Faces)

    pl = pv.Plotter(off_screen=True, window_size=[512,384])
    pl.add_mesh(Original, color=(1.0,0.0,0.0), show_edges=True, opacity=0.25)
    pl.add_mesh(Rotated, color=(0.0,0.0,1.0), show_edges=True, opacity=0.25)
    pl.add_points(GridPoints, color=(0.0,0.0,1.0), point_size=5)
    pl.add_points(RGridPoints, color=(1.0,0.0,0.0), point_size=5)
    pl.camera.roll = 0
    pl.camera.elevation = 30
    pl.camera.azimuth = 30
    pl.camera.zoom(1.0)
    pl.add_axes(viewport=(0,0,0.3,0.3))
    # pl.screenshot(FName.parent / (FName.name + '.png'))
    pl.show()

plot_corners_with_grid(Corners, R_Corners, Grid, R_Grid)

In [ ]:
Origin = Grid[0]
NPoints = round(MaxC[2] - MinC[2]) + 1

Steps = np.arange(0, round(NPoints))

Ray = np.tile(Origin,NPoints).reshape(NPoints,3)
Ray[:,2] = Ray[:,2] + Steps

Origin = R_Grid[0]

Steps = np.arange(0, round(NPoints)).reshape(-1, 1) * Direction

R_Ray = np.tile(Origin,NPoints).reshape(NPoints,3)
R_Ray = np.round(R_Ray + Steps).astype(int)

def plot_corners_with_grid_and_ray(Corners, RCorners, GridPoints, RGridPoints, Ray, R_Ray):
    
    """Plot binary array with mirrored rotated corners."""

    # Corners bounding box
    Faces = [
                [4, 0, 1, 2, 3],  # Bottom face
                [4, 4, 5, 6, 7],  # Top face
                [4, 0, 1, 5, 4],  # Side face 1
                [4, 1, 2, 6, 5],  # Side face 2
                [4, 2, 3, 7, 6],  # Side face 3
                [4, 3, 0, 4, 7],  # Side face 4
            ]
    
    Faces = np.hstack(Faces)

    # Create the box in PyVista
    Original = pv.PolyData(Corners, Faces)
    Rotated = pv.PolyData(RCorners, Faces)

    pl = pv.Plotter(off_screen=True, window_size=[512,384])
    pl.add_mesh(Original, color=(1.0,0.0,0.0), show_edges=True, opacity=0.25)
    pl.add_mesh(Rotated, color=(0.0,0.0,1.0), show_edges=True, opacity=0.25)
    pl.add_points(GridPoints, color=(0.0,0.0,1.0), point_size=5)
    pl.add_points(RGridPoints, color=(1.0,0.0,0.0), point_size=5)
    pl.add_lines(np.array([Ray[0], Ray[-1]]), color=(0.0,0.0,1.0), width=2)
    pl.add_lines(np.array([R_Ray[0], R_Ray[-1]]), color=(1.0,0.0,0.0), width=2)
    pl.camera.roll = 0
    pl.camera.elevation = 30
    pl.camera.azimuth = 30
    pl.camera.zoom(1.0)
    pl.add_axes(viewport=(0,0,0.3,0.3))
    # pl.screenshot(FName.parent / (FName.name + '.png'))
    pl.show()

plot_corners_with_grid_and_ray(Corners, R_Corners, Grid, R_Grid, Ray, R_Ray)

In [ ]:
intercept_lengths = []
intercept_counts = []
Rays = []
Valid = []

Steps = np.arange(0, round(NPoints)).reshape(-1, 1) * Direction

for Origin in R_Grid:

    # Build ray
    Ray = np.tile(Origin,NPoints).reshape(NPoints,3)
    Ray = np.round(Ray + Steps).astype(int)

    # Filter points within bounds of binary array
    Points = Ray[
        (Ray[:, 0] >= 0) & (Ray[:, 0] < X) &
        (Ray[:, 1] >= 0) & (Ray[:, 1] < Y) &
        (Ray[:, 2] >= 0) & (Ray[:, 2] < Z)
    ]

    # Skip if no valid points
    if Points.size == 0:
        continue

    # Store rays for plotting
    Rays.append(Ray)
    Valid.append(Points)

    # Get array values
    Values = Array[Points[:,0], Points[:,1], Points[:,2]]

    # Count intercepts
    intercepts = np.diff(Values)
    intercept_count = np.count_nonzero(intercepts != 0)

    # Store number of intercepts and length
    intercept_counts.append(intercept_count)
    intercept_lengths.append(np.linalg.norm(Points[-1] - Points[0]))

def plot_binary_array_with_ray_and_transparent_outside(Corners, Rays, Valid):
    
    """Plot binary array with ray inside bounds as solid blue, and outside as transparent."""

    # Corners bounding box
    Faces = [
                [4, 0, 1, 2, 3],  # Bottom face
                [4, 4, 5, 6, 7],  # Top face
                [4, 0, 1, 5, 4],  # Side face 1
                [4, 1, 2, 6, 5],  # Side face 2
                [4, 2, 3, 7, 6],  # Side face 3
                [4, 3, 0, 4, 7],  # Side face 4
            ]
    
    Faces = np.hstack(Faces)

    # Create the box in PyVista
    Original = pv.PolyData(Corners, Faces)

    pl = pv.Plotter(off_screen=True, window_size=[512,384])
    pl.add_mesh(Original, color=(0.0,0.0,0.0), show_edges=True, opacity=0.25)
    pl.camera.roll = 0
    pl.camera.elevation = 30
    pl.camera.azimuth = 30
    pl.camera.zoom(1.0)
    pl.add_axes(viewport=(0,0,0.3,0.3))
    for Ray, Points in zip(Rays, Valid):
            # Plot ray inside array bounds
        pl.add_lines(np.array([Ray[0], Ray[-1]]), color=(1.0,0.0,0.0), width=2)
        pl.add_lines(np.array([Points[0], Points[-1]]), color=(0.0,0.0,1.0), width=5)
    # pl.screenshot(FName.parent / (FName.name + '.png'))
    pl.show()

plot_binary_array_with_ray_and_transparent_outside(Corners, Rays[::100], Valid[::100])

In [ ]:

# Compute MIL for the mirrored direction
MIL = np.sum(intercept_lengths) / np.sum(intercept_counts)
MIL


In [ ]:
# Rotate corners to align z-axis with mirrored direction
R_Corners = rotate_to_align_with_direction(Corners, D, Center)

# Generate grid of ray origins in the xy-plane, spaced by `StepSize`
MinC, MaxC = R_Corners.min(axis=0), R_Corners.max(axis=0)
Xp = np.arange(0, MaxC[0], StepSize)
Xn = np.arange(-StepSize, MinC[0], -StepSize)
Yp = np.arange(0, MaxC[1], StepSize)
Yn = np.arange(-StepSize, MinC[1], -StepSize)
Xc = np.concatenate([Xn[::-1], Xp]).astype(int)
Yc = np.concatenate([Yn[::-1], Yp]).astype(int)

XGrid, YGrid = np.meshgrid(Xc, Yc)
Grid = np.column_stack((XGrid.ravel(),
                        YGrid.ravel(),
                        np.zeros(XGrid.size, int) + int(round(MinC[2]))))

# Rotate grid back to the original space
R_Grid = rotate_back_to_align_with_direction(Grid, D, Center)

# Compute steps to generate ray in the given direction
NPoints = round(MaxC[2] - MinC[2]) + 1

64